Импортируем библиотеки, которые будут использованы

In [ ]:
import requests as rq
import bs4
import json
import pandas as pd

Подключаемся к сайту по региону Краснодарский край и проверяем статус соединения:

In [ ]:
url = "https://auto.drom.ru/region23/premium/"
car_krasnodar = rq.get(url)
car_krasnodar.status_code

200

Подключение успешно

Через библиотеку bs4 знакомимся с данными

In [ ]:
soup_car = bs4.BeautifulSoup(car_krasnodar.text, 'html.parser')
soup_car

<!DOCTYPE html>

<html class="drom-notouch" lang="ru" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Спецразмещение в Краснодарском крае</title>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="drom.ru" name="copyright"/>
<meta content="#000000" name="theme-color"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta charset="utf-8"/>
<meta content='{"geor":77,"geoc":0,"id":27,"b":1,"bc":1,"charset":"utf-8","cf":{"category_id":1}}' name="candy.config"/>
<meta content="width=device-width" name="viewport"/>
<meta content="Частные объявления о продаже автомобилей на Drom.ru (Россия, Москва, Новосибирск, Владивосток и другие города)" name="description"/>
<meta content="авто, автомобили, форумы, автофорум, продажа б/у, пробег, подержанные, продажа авто, продажа автомобилей, отзывы, автовладельцы, объявления, цены, продажа, toyota, nissan, mitsubishi, honda, mazda, suzuki, subaru, audi, bmw, ford, opel, kia, hyundai, каталог" nam

 Перебирая и изучая различные теги, обратил внимание, что в одном из тегов title присутствуют теги (\<a href>\) со сылками на страницы отдельных авто, имеющие один и тот же класс: 'css-b5o66h'
 Для этого создаем пустой список, в который будем добавлять ссылки:

https://github.com/andrzejabramov/odin_colab/blob/main/img/img_link_list.png

In [ ]:
car_list = []

for car in soup_car.find_all('a', class_='css-b5o66h'):
  car_list.append(car.get('href'))

car_list

['https://auto.drom.ru/sochi/renault/sandero_stepway/130700713.html',
 'https://auto.drom.ru/krasnodar/renault/logan/882794966.html',
 'https://auto.drom.ru/krasnodar/skoda/octavia/196561247.html',
 'https://auto.drom.ru/krasnodar/nissan/pathfinder/745725628.html',
 'https://auto.drom.ru/krasnodar/volvo/xc90/129001622.html',
 'https://auto.drom.ru/krasnodar/nissan/qashqai/467742767.html',
 'https://auto.drom.ru/krasnodar/changan/uni-k/505697346.html',
 'https://auto.drom.ru/krasnodar/nissan/note/511862464.html',
 'https://auto.drom.ru/krasnodar/toyota/land_cruiser/550139036.html',
 'https://auto.drom.ru/krasnodar/volkswagen/tiguan/671247703.html',
 'https://auto.drom.ru/krasnodar/hyundai/solaris/170987759.html',
 'https://auto.drom.ru/krasnodar/citroen/c3_picasso/216327831.html',
 'https://auto.drom.ru/krasnodar/land_rover/freelander/251987606.html',
 'https://auto.drom.ru/krasnodar/skoda/octavia/281325327.html',
 'https://auto.drom.ru/plastunovskaya/hyundai/solaris/357765109.html',
 '

Попробуем вручную проверить работу ссылок по отдельным авто и посмотреть структуру этих html страниц

In [ ]:
new_url = 'https://auto.drom.ru/korzhevskiy/honda/freed_plus/672284294.html'
new_car = rq.get(new_url)
new_car.status_code

200

Изучая структуру страницы, ищем в поисковике слово price и находим интересный тег (\<script type "application/json">), включающий в себя несколько словарей, один из которых  имеет ключ name с маркой и моделью авто и ключ price с ценой авто

https://github.com/andrzejabramov/odin_colab/blob/main/img/img_tag.png

https://github.com/andrzejabramov/odin_colab/blob/main/img/img_price.png

Применим к страничке bs4 и выведем в консоль что получилось

In [ ]:
soup_new_car = bs4.BeautifulSoup(new_car.text, 'html.parser')

Вот эти словари:
{"@context":"https:\/\/schema.org","@type":"Car",__"name":"Honda Freed+"__,"brand":{"@type":"Brand","name":"Honda"},"image":{"@type":"ImageObject","url":"https:\/\/s11.auto.drom.ru\/photo\/v2\/kpOK7bFfqVb8Ew5orGHfVA2IqH90zis7ukMfbO7TrPJn1FVQcdNym5iXSUC6_fiC-mNW4ALRoR5CZEjH\/gen600.jpg"},"vehicleModelDate":"2023","description":"Honda Freed+ 2023 в Коржевском, 1.5 литра, пер.привод, цена 2550000 р., Обкатку машины, будете проходить Вы сами и обслуживать со 100% уверенностью тоже, автоматическая коробка","offers":{__"@type":"Offer","price":2550000__,"priceCurrency":"RUB","availability":"https:\/\/schema.org\/InStock","url":"https:\/\/auto.drom.ru\/korzhevskiy\/honda\/freed_plus\/672284294.html"},"mileageFromOdometer":{"@type":"QuantitativeValue","value":"34","unitCode":"KMT"},"aggregateRating":{"@type":"AggregateRating","ratingValue":9.3,"ratingCount":193,"bestRating":10},"model":"Freed+","vehicleIdentificationNumber":"388**************","itemCondition":"http:\/\/schema.org\/UsedCondition"}

In [ ]:
for new in soup_new_car.find_all('script', type="application/ld+json"):
  if 'offers' in new.text:
    print(json.loads(new.text)['name'], json.loads(new.text)['offers']['price'])


Honda Freed+ 2550000


Исследовав на нескольких авто, увидел, что эти словари имеют единый стиль и состав для многих авто, на основании чего принято решение данные брать именно отсюда

Создаем пустой дата фрейм со столбцами 'name_auto', 'price'

In [ ]:
df = pd.DataFrame(columns=['name_auto', 'price'])
df

,name_auto,price


Пишем результирующий код парсинга по заданию:  
1) проходим итерацией по ссылкам на страницы отдельных авто.   
2) применяем к странице bs4.    
3) находим тег script type="application/json" и выбираем данные   
4) Так как получаем три словаря, проходим по ним и выбираем тот, в котором есть ключ 'offers'.    
5) при нахождении формируем словарь из двух ключей 'name_auto', в который в качестве значения добавляем полученную из json, преобразованную в словарь марку и модель авто, а в ключ 'price' значение из пути json: 'offers' -> 'price'.  
6) Добавляем этот словарик в созданный data фрейм.  
7) Выводим дата фрейм в консоль.

In [25]:
for car in car_list:
  soup_new_car = bs4.BeautifulSoup(rq.get(car).text, 'html.parser')
  for new in soup_new_car.find_all('script', type="application/json"):
    if 'offers' in new.text:
      new_row = {
          'name_auto': json.loads(new.text)['name'],
          'price': json.loads(new.text)['offers']['price']
      }
      df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

df

,name_auto,price
0,Renault Sandero Stepway,890000
1,Renault Logan,600000
2,Skoda Octavia,1095000
3,Nissan Pathfinder,1325000
4,Volvo XC90,1595000
...,...,...
100,Hyundai Santa Fe,1095000
101,Kia Rio,1449000
102,Audi Q7,1695000
103,Land Rover Range Rover,9500000


<script type="application/ld+json"

{"@context":"https:\/\/schema.org","@type":"Car","name":"Лада Нива Легенд","brand":{"@type":"Brand","name":"Лада"},"image":{"@type":"ImageObject","url":"https:\/\/s12.auto.drom.ru\/photo\/v2\/21dj7mvWteEtbdsPCyTLedj-j1tLO2g-pOgdqcuH6wFZ5F4Vn3nG6kafAfV6Q29mo-KUxe-gjcz7LIyr\/gen600.jpg"},"vehicleModelDate":"2024","description":"Продажа Лада Niva Legend 2024 год в Краснодаре, мкпп, Продам НИВУ 2121 ЛЕГЕНД, джип\/suv 3 дв., комплектация 1.7 MT Luxe, бензин, б\/у, 4 вд, 1.7 литра","offers":{"@type":"Offer","price":959000,"priceCurrency":"RUB","availability":"https:\/\/schema.org\/InStock","url":"https:\/\/auto.drom.ru\/krasnodar\/lada\/niva_legend\/859205364.html"},"mileageFromOdometer":{"@type":"QuantitativeValue","value":"5000","unitCode":"KMT"},"aggregateRating":{"@type":"AggregateRating","ratingValue":8.4,"ratingCount":691,"bestRating":10},"model":"Нива Легенд","vehicleIdentificationNumber":"386**************","itemCondition":"http:\/\/schema.org\/UsedCondition"}

</script>